<a href="https://colab.research.google.com/github/ayaka-honda/lesson_ai/blob/master/tokyo_coffee_wordcloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# CSVファイルの読込
import pandas as pd
data=pd.read_csv('/content/drive/My Drive/Coffee/review.csv')
data.head()

,store_id,store_name,score,review
0,1,カフェ・ド・ランブル,3.76,この日は新橋周辺へ。同級生の烏丸大路に片思いするごく普通の女子高校生・塚本天満、その天満に片...
1,1,カフェ・ド・ランブル,3.76,日本の自家焙煎珈琲店のルーツであるコチラ。定温の専用倉庫で、10年以上保管・熟成した生豆を『...
2,1,カフェ・ド・ランブル,3.76,2019/6月下旬。 自分の中の銀座3大カフェの1軒でした。 勝手にみゆき館、壹...
3,1,カフェ・ド・ランブル,3.76,銀ブラの語源は銀座でブラジルコーヒーを飲むことであるという説をある珈琲店が主張しており、しか...
4,1,カフェ・ド・ランブル,3.76,喫煙可で狭くて混んでいて､寛げませんが他ではあまり味わえないコーヒーが頂けます。昔は珈琲通で...


In [0]:
# 日本語が使えるように日本語フォントの設定
!apt-get -y install fonts-ipafont-gothic
!ls /usr/share/fonts/opentype/ipafont-gothic/ipagp.ttf

# 正規化と形態素解析のライブラリをインポート
!pip install janome
from janome.charfilter import *
from janome.analyzer import Analyzer
from janome.tokenizer import Tokenizer
from janome.tokenfilter import *
import re
# WordCloudのライブラリをインポート
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
# ファイル保存名用に'store_name'だけのリストを取得
# store idの重複を削除
store_original=data[~data['store_id'].duplicated()]
# print(store_original)

# 'store_name'のリスト作成、element数の確認と重複店名の抽出
store_uni=store_original['store_name']
print('重複:',store_uni[store_uni.duplicated()])
print('element:',len(store_uni))

store_original[store_original['store_name']=='十一房珈琲店']

重複: 2408    十一房珈琲店
Name: store_name, dtype: object
element: 580


,store_id,store_name,score,review
140,8,十一房珈琲店,3.71,【2019/11/15】 十一房珈琲店 この日は仕事が終わった後、浜松町で軽い食事した後...
2408,123,十一房珈琲店,3.37,2016.9月 希少豆で淹れた珈琲をいただく。ボリビア（￥８００）生豆のまま長期間寝かせた、...


In [0]:
# 重複店名の変更
store_original['store_name'][2408]='十一房珈琲店 自由が丘'

In [28]:
# 変更を確認
store_original[store_original['store_name']=='十一房珈琲店 自由が丘']

,store_id,store_name,score,review
2408,123,十一房珈琲店 自由が丘,3.37,2016.9月 希少豆で淹れた珈琲をいただく。ボリビア（￥８００）生豆のまま長期間寝かせた、...


In [0]:
store_uni=store_original['store_name']

In [30]:
# 保存のファイル名に影響のありそうな店名を補完
store=[]
for strip in store_uni:
  my_strip=re.sub(r'[\/ \\u]', ' ', strip)
  store.append(my_strip)
print('element:',len(store)) 
print('変更された店名:\n',store_uni[store_uni!=store])

element: 580
変更された店名:
 460         Turret Coffee
880       東向島珈琲店 pua mana
1720    YOUR DAILY/COFFEE
2073           sunday zoo
2448       百塔珈琲 Shimofuri
6291        Caffe’ Lumino
Name: store_name, dtype: object


In [31]:
# 店ごとのレビューを取得する為、indexを'store_id'に変更
data.set_index('store_id', inplace=True)
data

,store_name,score,review
store_id,,,
1,カフェ・ド・ランブル,3.76,この日は新橋周辺へ。同級生の烏丸大路に片思いするごく普通の女子高校生・塚本天満、その天満に片...
1,カフェ・ド・ランブル,3.76,日本の自家焙煎珈琲店のルーツであるコチラ。定温の専用倉庫で、10年以上保管・熟成した生豆を『...
1,カフェ・ド・ランブル,3.76,2019/6月下旬。 自分の中の銀座3大カフェの1軒でした。 勝手にみゆき館、壹...
1,カフェ・ド・ランブル,3.76,銀ブラの語源は銀座でブラジルコーヒーを飲むことであるという説をある珈琲店が主張しており、しか...
1,カフェ・ド・ランブル,3.76,喫煙可で狭くて混んでいて､寛げませんが他ではあまり味わえないコーヒーが頂けます。昔は珈琲通で...
...,...,...,...
580,鳴嶋珈琲,3.07,○撮影了承済 (LUMIX G5)デザミランチの帰り際に見つけた鳴嶋珈琲。つい数日前に新規開...
580,鳴嶋珈琲,3.07,２９．１２．８ 初訪【来店動機】プリムーロ(店名)から当店も旨いと言う情報を得たので…。【雰...
580,鳴嶋珈琲,3.07,上石神井駅から徒歩１分。いつの間にか、珈琲屋さんが出来ていました。デザミでランチの後に、２度...


In [0]:
#　店舗ごとに口コミを正規化、形態素解析して単語リストを作成、可視化
def coffee():
  texts=data['review'][i]

  # janomeのAnalyzerを使うことで、文の分割と単語の正規化をまとめて行うことができる
  # 文に対する処理のまとめ
  char_filters = [UnicodeNormalizeCharFilter(),         # UnicodeをNFKC(デフォルト)で正規化
                  RegexReplaceCharFilter('\(', ''),     # (を削除
                  RegexReplaceCharFilter('\)', '')      # )を削除
                  ]
  # 単語に分割
  tokenizer = Tokenizer()

  #
  #  ひらがな・カタカナ・英数字の一文字しか無い単語は削除
  #
  class OneCharacterReplaceFilter(TokenFilter):

      def apply(self, tokens):
          for token in tokens:
              # 上記のルールの一文字制限で引っかかった場合、その単語を無視
              if re.match('^[あ-んア-ンa-zA-Z0-9ー]$', token.surface):
                  continue

              yield token


  # 単語に対する処理のまとめ
  token_filters = [CompoundNounFilter(),                          # 名詞が連続する場合は複合名詞にする
                  POSKeepFilter(['名詞', '形容詞', '副詞']),      # 名詞・形容詞・副詞のみを取得する
                  LowerCaseFilter(),                              # 英字は小文字にする
                  OneCharacterReplaceFilter()                     # 一文字しか無いひらがなとカタカナと英数字は削除
                  ]

  analyzer = Analyzer(char_filters, tokenizer, token_filters)

  tokens_list = []
  raw_texts = []
  for text in texts:
      # 文を分割し、単語をそれぞれ正規化する
      text_ = [token.base_form for token in analyzer.analyze(text)]
      if len(text_) > 0:
          tokens_list.append([token.base_form for token in analyzer.analyze(text)])
          raw_texts.append(text)


  # 正規化された際に一文字もない文の削除後の元テキストデータ
  raw_texts = [text_+'\n' for text_ in raw_texts]

  # 単語リストの作成
  words = []
  for text in tokens_list:
      words.extend([word+'\n' for word in text if word != ''])

  # 単語リストをstring型に変換して繋げる
  review_text = ' '.join(words).replace('\n', '')

  
  fpath = "/usr/share/fonts/opentype/ipafont-gothic/ipagp.ttf"

  # ストップワードの設定
  # ここで設定した単語はWord Cloudに表示されない
  stop_words = [u'コーヒー', u'珈琲', u'カフェ', u'美味しい', u'雰囲気', u'こちら', u'てる', u'いる', u'なる', u'れる', u'する', u'ある', u'こと', u'これ', u'さん', u'して', \
              u'くれる', u'やる', u'くださる', u'そう', u'せる', u'した', u'思う', \
              u'それ', u'ここ', u'ちゃん', u'くん', u'', u'て', u'に', u'を', u'は', u'の', u'が', u'と', u'た', u'し', u'で', \
              u'ない', u'も', u'な', u'い', u'か', u'ので', u'よう', u'', u'もの', u'もつ']

  wordcloud = WordCloud(background_color="white",
                      font_path=fpath,
                      width=900,
                      height=500,
                      stopwords=set(stop_words)).generate(review_text)

  my_path='/content/drive/My Drive/Coffee/wordcloud/'
  plt.figure(figsize=(10, 8))
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout()
  plt.savefig(my_path + store[i-1]+'.png')
  plt.close()
  # plt.show()

In [0]:
for i in range(1,580):
  coffee()